各階段設計思路
最後更新：2022/06/18

階段二、設計系統(初始設定)

接下來在系統的運行中，有些部分是需要先定義的。
例如函式、數值及空串列。但如果每次都是要用到的時候才給定義的話，版面或許會有些凌亂。因此我選擇將它納入進最前面的部分，也就是程式運行篩選迴圈前，就先弄完定義，這樣空間應該會整潔不少。
下列我們看系統初始設定的部分。

首先，我們在運行時，會使用到一些原本Python不會預設附帶的套件，因此我們需要利用pip指令來安裝。
以Google Colab為例，我們會需要額外安裝twstock與yfinance。
至於一般有預設的套件則是直接import即可，不需重複安裝。
由於套件名稱有點冗長，於是我也另外給定對應的代號。

In [ ]:
!pip install twstock
!pip install yfinance
import twstock as t 
import pandas as p 
from bs4 import BeautifulSoup 
import requests 
import time as ti
import os as o

接下來是定義函式的部分。
這個系統中我們運用到兩個函式，一個是網頁查詢使用，另一個是Line Notify通知。
由於本身對於api串接的熟悉度不是很好，因此這邊參考他人的程式代碼:(https://jeffwen0105.com/python_linenotify/)

In [ ]:
def web(url):#定義網頁查詢函式
    source = requests.get(url, headers={'Connection':'close'}) #連線到指定的網站
    #因為股票要篩選好幾千個，若我們一直開啟新網頁而不關掉已使用完畢的頁面，電腦資源會被浪費
    soup = BeautifulSoup(source.content, 'lxml', from_encoding='utf-8') #讀取這個網頁的內容
    return soup #最終結果:回傳網頁內容

def post_data(message, token):#Line notify傳訊息
    try:
        url = "https://notify-api.line.me/api/notify"#api網址
        headers = {
            'Authorization': f'Bearer {token}'
        }
        payload = {
            'message': message
        }
        response = requests.request(
            "POST",
            url,
            headers=headers,
            data=payload
        )
        if response.status_code == 200:
            print(f"Success -> {response.text}")
    except Exception as _:
        print(_)#狀態

接下來是取得最新股票資料。
在這邊我們利用pandas套件去讀取網頁內容，因上市及上櫃股為不同網頁因此這邊要取得兩次資料。
因為我們只需要 "有價證券代號及名稱" 的這部分資訊，因此我們單獨把這部分拉來製表後，再替代掉原先的表格。
接下來我們只需要重複上市股的取得動作，但把網址改成上櫃股的網址，這樣我們就完成取得上市股跟上櫃股的最新清單步驟了！

In [ ]:
#####取得上市股#####
data = p.read_html('http://isin.twse.com.tw/isin/C_public.jsp?strMode=2', encoding='big5hkscs', header=0)[0]
data = p.DataFrame(data["有價證券代號及名稱"])
data = data["有價證券代號及名稱"].astype(str)

#####取得上櫃股#####
data2 = p.read_html('http://isin.twse.com.tw/isin/C_public.jsp?strMode=4', encoding='big5hkscs', header=0)[0]
data2 = p.DataFrame(data2["有價證券代號及名稱"])
data2 = data2["有價證券代號及名稱"].astype(str)

接下來是一些基本定義。
因為後續判斷條件中，我在作者的判斷條件上另外多加 "是否為當沖個股" 的判斷條件。
因此我也在前期就先給定好，後續可以直接調用。

In [ ]:
#####取得可當沖股#####
TW50 = [1101, 1216, 1301, 1303, 1326, 1590, 2002, 2207, 2227, 2303, 2308, 2317, 2327, 2330, 2352, 2357, 2379, 2382, 2395, 2408, 2409, 2412, 2454, 2603, 2609, 2610, 2615, 2801, 2880, 2881, 2882, 2883, 2884, 2885, 2886, 2887, 2891, 2892, 2912, 3008, 3034, 3037, 3045, 3711, 4904, 5871, 5876, 5880, 6415, 6505, 8046, 8454, 9910]
RICH50 = [1565, 1785, 3081, 3105, 3152, 3211, 3218, 3227, 3228, 3260, 3264, 3293, 3324, 3374, 3529, 3552, 3680, 3707, 4105, 4123, 4128, 4736, 4966, 5274, 5289, 5347, 5351, 5371, 5425, 5483, 6104, 6121, 6138, 6147, 6182, 6274, 6411, 6488, 6510, 6547, 6548, 6561, 6732, 6741, 8069, 8086, 8255, 8299, 8358, 8436]
MID100A = [1102,1210,1227,1229,1402,1434,1440,1451,1476,1477,1504,1560,1605,1717,1718,1722,1802,1907,2006,2014,2023,2027,2049,2105,2201,2301,2312,2313,2324,2337,2344,2347,2353,2354,2356,2360,2371,2376,2377,2385,2441,2449,2458,2474,2478,2492,2498,2542,2606,2618,2633,2637,2809,2812,2834,2845,2888,2889,2890,2915,3023,3035,3036,]
MID100B = [3044,3051,3189,3231,3406,3443,3481,3532,3533,3653,3661,3702,3714,4137,4919,4938,4958,4961,5269,5522,6005,6116,6176,6213,6239,6271,6409,6531,6592,6669,6719,6770,6781,8464,9904,9914,9917,9921,9941,9945]
Ulti_list = RICH50  + MID100A + MID100B + TW50 

#####創建空串列#####
stocklist=[]#篩選股票清單
stock28list = []#符合條件清單
amu28list = []#符合精選條件清單
ulti28list = []#符合當沖精選條件清單

#####代數預設指定#####
d1s = 1 #上市股開始index碼
d1e = 968 #上市股結束index碼
d2s = 7408 #上櫃股開始index碼
d2e = 8205 #上櫃股結束index碼

因為股票會有新上市或下市的情況，因此接下來是系統請使用者檢查在證交所網頁中，上市/櫃股的index始終代數是否有變動。
這邊採用input方式，這樣可以減少在代碼中尋找參數設定的時間。
因為會牽扯到系統輸出的問題，為求版面整潔下面先採用了os套件來進行清屏。

In [ ]:
fast_start = int(input("冷啟動請輸入0|快啟動請輸入1:"+"\n上市股始/終:"+str(d1s)+"/"+str(d1e)+"\n上櫃股始/終:"+str(d2s)+"/"+str(d2e)+"\n"))

#####檢查上市股代數變動#####
check_data_index = int(input("請問上市股index是否有改變?(0無1有)"+"\n目前預設起始/終止值為:"+str(d1s)+"/"+str(d1e)))

if check_data_index == 0 and fast_start == 0:
  o.system( 'cls' )
elif check_data_index == 1 and fast_start == 0:
  d1s = int(input("請輸入上市股起始index:"))
  o.system( 'cls' )
  d1e = int(input("請輸入上市股終止index:"))
  o.system( 'cls' )
  
#####檢查上櫃股代數變動#####
check_data2_index = int(input("請問上櫃股index是否有改變?(0無1有)"+"\n目前預設起始/終止值為:"+str(d2s)+"/"+str(d2e)))

if check_data2_index == 0 and fast_start == 0:
  o.system( 'cls' )
elif check_data2_index == 1 and fast_start == 0:
  d2s = int(input("請輸入上櫃股起始index:"))
  o.system( 'cls' )
  d2e = int(input("請輸入上櫃股終止index:"))
  o.system( 'cls' )

接下來使用for迴圈將前面，使用者指定之index值所對應的內容存到list當中

In [ ]:
#####匯入上市股至篩選股票清單#####
for x in data.iloc[d1s:d1e]:#個別抓資料
  try:
    stockid = x
    stocklist.append(stockid)#匯入清單
  except:
    continue
#####匯入上櫃股至篩選股票清單#####
for z in data2.iloc[d2s:d2e]:#個別抓資料
  try:
    stockid = z
    stocklist.append(stockid)#匯入清單
  except:
    continue